In [2]:
import paperdecoding
import numpy as np
import pandas as pd
import glob
import re

# All papers and reports to text #

In [14]:
for path in glob.glob('sa2papers/*.pdf'):
    pages = paperdecoding.gettextpages(path)
    pages = [page for page in pages if page is not None]
    outpath = path.replace('\\','_txt\\').replace('pdf','txt')
    paperdecoding.pages_totext(pages,outpath)

# Parse and tidy questions #


In [2]:
all_papers = [paperdecoding.pagesfromtext(x) for x in glob.glob('sa2papers_txt/*[0-9].txt')]

In [12]:
paperdates = [x[14:20] for x in glob.glob('sa2papers_txt/*[0-9].txt')]

Regex out the questions - I'm not sure if sa2 has any single-part questions (ie. questions that don't have a [total x] at the end).

In [77]:
qs = [re.findall('\n\d .+?\[Total \d+\]', '\n'.join(['']+p[1:]), re.DOTALL) for p in all_papers]
qs[25].pop(3)
qs[25].pop(1)
marks = [[float(re.findall('\[Total (\d+?)\]',q)[0]) for q in q_i] for q_i in qs]
qs[13].append(re.findall('\n3 .+?\[\d+\]', '\n'.join(['']+all_papers[13][1:]), re.DOTALL)[0])
marks[13].append(10)
bd = [[[float(mark) for mark in re.findall('\[(\d+?)\]',q)] for q in q_i] for q_i in qs]
bd[26][0]=[3.0, 6.0, 10.0, 8.0, 7.0, 6.0, 8.0]
qnos = [[int(s[1]) for s in q_i] for q_i in qs]

In [67]:
[[sum(x) for x in y] for y in bd]

[[32.0, 68.0],
 [26.0, 30.0, 44.0],
 [55.0, 45.0],
 [30.0, 45.0, 25.0],
 [68.0, 32.0],
 [70.0, 30.0],
 [43.0, 34.0, 23.0],
 [75.0, 25.0],
 [48.0, 52.0],
 [27.0, 29.0, 44.0],
 [57.0, 43.0],
 [35.0, 28.0, 37.0],
 [67.0, 33.0],
 [39.0, 51.0, 10.0],
 [37.0, 63.0],
 [50.0, 50.0],
 [60.0, 24.0, 16.0],
 [48.0, 52.0],
 [48.0, 52.0],
 [66.0, 34.0],
 [44.0, 56.0],
 [50.0, 50.0],
 [50.0, 50.0],
 [60.0, 40.0],
 [44.0, 56.0],
 [44.0, 56.0],
 [51.0, 52.0],
 [48.0, 52.0],
 [34.0, 66.0],
 [53.0, 47.0],
 [31.0, 29.0, 40.0],
 [29.0, 28.0, 43.0],
 [21.0, 29.0, 50.0]]

In [78]:
for i, q_i in enumerate(qs):
    for j, q_ij in enumerate(qs[i]):
        if marks[i][j] != sum(bd[i][j]):
            print(i,j)

In [82]:
q_dictlist = [{'full_date': paperdates[i],
'question_no': qnos[i][j],
'total_marks': marks[i][j],
'breakdown': bd[i][j],
'question_text': q_ij} for i, q_i in enumerate(qs) for j, q_ij in enumerate(qs[i])]

In [83]:
qdf = pd.DataFrame(q_dictlist)
qdf

,full_date,question_no,total_marks,breakdown,question_text
0,200504,1,32.0,"[6.0, 8.0, 18.0]",\n1 A UK life insurance company sells mainly ...
1,200504,2,68.0,"[5.0, 11.0, 5.0, 9.0, 12.0, 26.0]",\n2 Company A is a small UK mutual life insur...
2,200509,1,26.0,"[12.0, 6.0, 8.0]",\n1 A UK bank is about to launch an equity re...
3,200509,2,30.0,"[16.0, 14.0]",\n2 A UK life insurance company currently sel...
4,200509,3,44.0,"[15.0, 6.0, 23.0]",\n3 A UK mutual life insurance company distri...
...,...,...,...,...,...
71,202009,2,28.0,"[3.0, 3.0, 4.0, 7.0, 11.0]",\n2 \n(i) State the three lines of defence a ...
72,202009,3,43.0,"[13.0, 17.0, 13.0]",\n3 \nA small life insurance company runs a wi...
73,202104,1,21.0,"[6.0, 5.0, 10.0]",\n1 A life insurance company sells a unit-lin...
74,202104,2,29.0,"[3.0, 14.0, 6.0, 6.0]",\n2 A life insurance company started writing ...


# Parse and tidy answers

In [3]:
all_reports = [paperdecoding.pagesfromtext(x) for x in glob.glob('sa2papers_txt/*[0-9]report.txt')]

In [8]:
#print(all_reports[0][1])

In [9]:
reportdates = [paperdecoding.getdatefrompaper(report) for report in all_reports]

In [10]:
rqnos = [re.split(r'\nQ?(\d) ', ''.join(report))[1::2] for report in all_reports]
rqs = [re.split(r'\nQ?(\d) ', ''.join(report))[2::2] for report in all_reports]

In [12]:
rqs[0][1]

' (i)  (a)  These regulations give powers to some regulators (e.g. the FSA) and \nconsumer bodies to challenge companies that use unfair terms in their \nstandardised consumer contracts.      \nThe regulations do not offer redress to individual consumers.  If the \npolicyholder wishes to raise a complaint under these regulations, then \nhe would have to do so via another body, such as the Financial \nOmbudsman Service.  However, the first step is for Company A to \naddress the issues raised by the policyholder and see whether they can \nbe resolved satisfactorily.      \nThe general test of whether a term is unfair is based on whether it \ncould give significant advantage to the company that could cause \ndetriment to the consumers.  It is not the case here that Company A \nobtains significant advantage from the terms of the with profits bonds \nthat cause detriment to consumers.  In fact the terms aim to protect \npolicyholders, as will be explained later in the response.      \nExamp